# Глубинное обучение для текстовых данных, ФКН ВШЭ
## Домашнее задание 3: Уменьшение размеров модели
### Оценивание и штрафы

Максимально допустимая оценка за работу — __10 баллов__.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн 24.10.25 23:59__ \
__Жесткий дедлайн 26.10.25 23:59__

### О задании

В этом задании вам предстоит научиться решать задачу Named Entity Recognition (NER) на самом популярном датасете – [CoNLL-2003](https://paperswithcode.com/dataset/conll-2003). В вашем распоряжении будет предобученный BERT, который вам необходимо уменьшить с минимальными потерями в качестве до размера 20М параметров. Для этого вы самостоятельно реализуете факторизацию эмбеддингов, дистилляцию, шеринг параметров и так далее.

В этом задании вам придется проводить довольно много экспериментов, поэтому мы рекомендуем не писать весь код в тетрадке, а завести разные файлы для отдельных логических блоков и скомпоновать все в виде проекта. Это позволит вашему ноутбуку не разрастаться и сильно облегчит задачу и вам, и проверяющим. Так же постарайтесь логгировать все ваши эксперименты в wandb, чтобы ничего не потерялось.

### Оценивание
Оценка за это домашнее задание будет формироваться из оценки за __задания__ и за __отчет__, в котором от вас требуется написать о проделанной работе. За отчет можно получить до 2-х баллов, однако в случае отсутствия отчета баллы за соответствующие задания не будут ставиться. Задания делятся на две части: _номерные_ и _на выбор_. За _номерные_ можно получить в сумме 6 баллов, за задания _на выбор_ можно получить до 14. То есть за все дз можно получить 22 балла (но не радуйтесь рано, это не так просто). Все, что вы наберете свыше 10, будет считаться бонусами.


### О датасете

Named Entity Recognition – это задача классификации токенов по классам сущностей. В CoNLL-2003 для именования сущностей используется маркировка **BIO** (Beggining, Inside, Outside), в которой метки означают следующее:

- *B-{метка}* – начало сущности *{метка}*
- *I-{метка}* – продолжнение сущности *{метка}*
- *O* – не сущность

Существуют так же и другие способы маркировки, например, BILUO. Почитать о них можно [тут](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging)) и [тут](https://www.youtube.com/watch?v=dQw4w9WgXcQ).

Всего в датасете есть 9 разных меток.
- O – слову не соответствует ни одна сущность.
- B-PER/I-PER – слово или набор слов соответстует определенному _человеку_.
- B-ORG/I-ORG – слово или набор слов соответстует определенной _организации_.
- B-LOC/I-LOC – слово или набор слов соответстует определенной _локации_.
- B-MISC/I-MISC – слово или набор слов соответстует сущности, которая не относится ни к одной из предыдущих. Например, национальность, произведение искусства, мероприятие и т.д.

Приступим!

Начнем с загрузки и предобработки датасета.

In [2]:
!pip3 install datasets==3.4.1 transformers tensorflow-datasets==4.9.8 vega-datasets==0.9.0 seqeval==1.2.2 optuna==4.2.1 evaluate==0.4.3 "protobuf<6.30" comet_ml trl -q

In [3]:
from typing import Union
import datasets
import trl

In [4]:
from kaggle_secrets import UserSecretsClient
import os

api_keys = UserSecretsClient()

os.system(f"huggingface-cli login --token {api_keys.get_secret('huggingface-cli')}")
# os.system(f"wandb login {api_keys.get_secret('wandb')}")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `huggingface-cli` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `huggingface-cli`


⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.


0

In [5]:
from datasets import load_dataset

dataset = load_dataset("eriktks/conll2003", trust_remote_code=True)

dataset = dataset.remove_columns(["id", "pos_tags", "chunk_tags"])
dataset

README.md: 0.00B [00:00, ?B/s]

conll2003.py: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3453
    })
})

In [6]:
dataset['train'][0]

{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [7]:
label_names = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
label2id = {label: i for i, label in enumerate(label_names)}

In [8]:
words = dataset["train"][0]["tokens"]
labels = dataset["train"][0]["ner_tags"]

for i in range(len(words)):
    print(f'{words[i]}\t{label_names[labels[i]]}')

EU	B-ORG
rejects	O
German	B-MISC
call	O
to	O
boycott	O
British	B-MISC
lamb	O
.	O


### Предобработка

На протяжении всего домашнего задания мы будем использовать _cased_ версию BERT, то есть токенизатор будет учитывать регистр слов. Для задачи NER регистр важен, так как имена и названия организаций или предметов искусства часто пишутся с большой буквы, и будет глупо прятать от модели такую информацию.

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

При токенизации слова могут разделиться на несколько токенов (как слово `Fischler` из примера ниже), из-за чего появится несоответствие между числом токенов и меток. Это несоответствие нам придется устранить вручную.

In [10]:
example = dataset["train"][12]
words = example["tokens"]
tags = [label_names[t] for t in example["ner_tags"]]
tokenized_text = tokenizer(example["tokens"], is_split_into_words=True)

print('Слова: ', words)
print('Токены:', tokenized_text.tokens())
print('Метки:', tags)

Слова:  ['Only', 'France', 'and', 'Britain', 'backed', 'Fischler', "'s", 'proposal', '.']
Токены: ['[CLS]', 'Only', 'France', 'and', 'Britain', 'backed', 'Fi', '##sch', '##ler', "'", 's', 'proposal', '.', '[SEP]']
Метки: ['O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'O', 'O', 'O']


__Задание 1 (1 балл).__ Токенизируйте весь датасет и для каждого текста выравните токены с метками так, чтобы каждому токену соответствовала одна метка. При этом важно сохранить нотацию BIO. И не забудьте про специальные токены! Должно получиться что-то такое:

In [11]:
# your code here
SPECIAL_TOKENS_LABEL = -100

def align_labels_with_tokens(ner_tags: list[int], tokenized_text):
    tokens = tokenizer.convert_ids_to_tokens(tokenized_text['input_ids'])
    special_tokens = tokenizer.all_special_ids

    aligned_tags = []
    
    # declare vars if you needed
    # YOUR CODE IS HERE
    prev_entity = 'O'
    
    for token_id, (token, word_id) in enumerate(zip(tokens, tokenized_text.words())):
        # special token
        if tokenized_text['input_ids'][token_id] in special_tokens or word_id is None:
            aligned_tags.append(SPECIAL_TOKENS_LABEL)
            continue
        
        # same word
        if token.startswith('##'):
            if prev_entity == 'O':
                aligned_tags.append(prev_entity)
            else:
                aligned_tags.append('I-' + prev_entity)
            continue
        
        # new word
        cur_entity = label_names[ner_tags[word_id]]

        if cur_entity == 'O':
            aligned_tags.append(cur_entity)
        else:
            cur_entity = cur_entity[2:]
            if cur_entity == prev_entity:
                aligned_tags.append('I-' + cur_entity)
            else:
                aligned_tags.append('B-' + cur_entity)
        prev_entity = cur_entity
        
    aligned_labels = [label if label == SPECIAL_TOKENS_LABEL else label2id[label] for label in aligned_tags]

    return aligned_labels

In [12]:
aligned_labels = align_labels_with_tokens(example["ner_tags"], tokenized_text)
tags = [label_names[t] if t > -1 else t for t in aligned_labels]
print("Выровненные метки:", aligned_labels)
print("Выровненные названия меток:", tags)

Выровненные метки: [-100, 0, 5, 0, 5, 0, 1, 2, 2, 0, 0, 0, 0, -100]
Выровненные названия меток: [-100, 'O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'O', -100]


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:376: FutureWarning: `BatchEncoding.words()` property is deprecated and should be replaced with the identical, but more self-explanatory `BatchEncoding.word_ids()` property.
  warnings.warn(


### Метрика

Для оценки качества NER обычно используют F1 меру с микро-усреднением. Мы загрузим ее из библиотеки `seqeval`. Функция `f1_score` принимает два 2d списка с правильными и предсказанными метками, записаными текстом, и возвращает для них значение F1. Вы можете использовать ее с параметрами по умолчанию.

In [28]:
from seqeval.metrics import f1_score
import numpy as np
import numpy.typing as npt
import evaluate

metric = evaluate.load("seqeval")

In [153]:
def compute_metrics(
    eval_preds: tuple[npt.NDArray, list[list[int]]],
) -> dict[str, float]:
    """
    Compute metrics on given batch.

    Args:
        eval_preds: tuple with the following contents:
            - logits: array of shape `(batch_size, seq_length, n_classes)`,
            - labels: list of lists, i.e. matrix of shape `(batch_size, seq_length)`
    Returns:
        metrics: dictionary with metrics calculated over all classes:
            - precision
            - recall
            - f1
            - accuracy
    """
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # YOUR CODE IS HERE

    # Remove ignored index (special tokens) and convert to labels
    labels = np.asarray(labels)
    batch_size = labels.shape[0]
    pred_labels = []
    true_labels = []
    for i in range(batch_size):

        local_true = predictions[i]
        local_pred = labels[i]

        local_true = local_true[local_pred != SPECIAL_TOKENS_LABEL]
        local_pred = local_pred[local_pred != SPECIAL_TOKENS_LABEL]

        local_true_str = np.full(local_true.shape, 'O', dtype='str')
        local_pred_str = np.full(local_pred.shape, 'O', dtype='str')

        for label_, id_ in label2id.items():
            local_true_str[local_true == id_] = label_
            local_pred_str[local_pred == id_] = label_
        true_labels.append(local_true_str.tolist()[:len(local_pred_str)])
        pred_labels.append(local_pred_str.tolist())
    
    all_metrics = metric.compute(predictions=pred_labels,
                                references=true_labels)

    return {
        "precision": all_metrics['overall_precision'],
        "recall": all_metrics['overall_recall'],
        "f1": all_metrics['overall_f1'],
        "accuracy": all_metrics['overall_accuracy'],
    }

Особенность подсчета F1 для NER заключается в том, что в некоторых ситуациях неправильные ответы могут засчитываться как правильные. Например, если модель предсказала `['I-PER', 'I-PER']`, то мы можем догадаться, что на самом деле должно быть `['B-PER', 'I-PER']`, так как сущность не может начинаться с `I-`. Функция `f1_score` учитывает это и поэтому работает только с текстовыми представлениями меток.

### Модель

В качестве базовой модели мы возьмем `bert-base-cased`. Как вы понимаете, он не обучался на задачу NER. Поэтому прежде чем приступать к уменьшению размера BERT, его необходимо дообучить.

__Задание 2 (1 балл)__ Дообучите `bert-base-cased` на нашем датасете с помощью обычного fine-tuning. У вас должно получиться хотя бы 0.9 F1 на тестовой выборке. Заметьте, что чем выше качество большой модели, тем лучше будет работать дистиллированный ученик. Для обучения можно использовать `Trainer` из Hugging Face.

In [30]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(label_names))

print('Число параметров:', sum(p.numel() for p in model.parameters()))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Число параметров: 107726601


In [31]:
api_keys.get_secret('huggingface-cli')

from huggingface_hub import notebook_login

api_keys = UserSecretsClient()
os.system(f"huggingface-cli login --token {api_keys.get_secret('huggingface-cli')}")
notebook_login()

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `huggingface-cli` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `huggingface-cli`


In [32]:
# your code here
SPECIAL_TOKENS_LABEL = -100

def tokenize_dataset(ner_tags, tokens, tokenizer):
    tokenized_text = tokenizer(tokens, is_split_into_words=True)

    aligned_labels = align_labels_with_tokens(ner_tags, tokenized_text)
    token_ids = tokenized_text["input_ids"]

    return {
        "input_ids": token_ids,
        "labels": aligned_labels
    }

In [33]:
def tokenize_ner_dataset(
    dataset: Union[datasets.DatasetDict, datasets.Dataset], tokenizer: AutoTokenizer
):
    return dataset.map(
        function=tokenize_dataset,
        fn_kwargs={"tokenizer": tokenizer},
        input_columns=["ner_tags", "tokens"],
        remove_columns=(
            dataset["train"] if "train" in dataset else dataset
        ).column_names,
    )

In [34]:
tokenized_dataset = tokenize_ner_dataset(dataset, tokenizer)
tokenized_dataset

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:376: FutureWarning: `BatchEncoding.words()` property is deprecated and should be replaced with the identical, but more self-explanatory `BatchEncoding.word_ids()` property.
  warnings.warn(


Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 3453
    })
})

In [35]:
# your code here
from transformers import DataCollatorForTokenClassification
from transformers import TrainingArguments
from transformers import Trainer

os.environ["WANDB_DISABLED"] = "true"
os.environ["COMET_LOG_ASSETS"] = "false" #todo

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer,
    padding=True,
    max_length=512,  # Максимальная длина последовательности
    pad_to_multiple_of=8,  # Дополнssение до кратного 8 (для эффективности на GPU)
    label_pad_token_id=SPECIAL_TOKENS_LABEL,  # Игнорировать padding в loss функции
    return_tensors="pt"
)

args = TrainingArguments(
    "data/bert-finetuned-ner",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    # report_to=["comet_ml"],
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=tokenizer,
)

trainer.train()

comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.230400,0.065012,0.967834,0.958952,0.963373,0.991979
2,0.045400,0.058634,0.974402,0.968368,0.971376,0.993009
3,0.024300,0.057117,0.974571,0.969347,0.971952,0.993112


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=2634, training_loss=0.07735684575143871, metrics={'train_runtime': 562.0501, 'train_samples_per_second': 74.945, 'train_steps_per_second': 4.686, 'total_flos': 1125329365567296.0, 'train_loss': 0.07735684575143871, 'epoch': 3.0})

In [36]:
def test_metrics(cur_trainer):
    print('TEST metrics')
    preds = cur_trainer.predict(tokenized_dataset['test'])
    return compute_metrics((preds.predictions, preds.label_ids))

In [37]:
test_metrics(trainer)

TEST metrics


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'precision': 0.9621535181236673,
 'recall': 0.9414116828929068,
 'f1': 0.9516695957820738,
 'accuracy': 0.9846519909865902}

In [38]:
del model

### Факторизация матрицы эмбеддингов

Можно заметить, что на данный момент матрица эмбеддингов занимает $V \cdot H = 28996 \cdot 768 = 22.268.928$ параметров. Это aж пятая часть от всей модели! Давайте попробуем что-то с этим сделать. В модели [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) предлагается факторизовать матрицу эмбеддингов в произведение двух небольших матриц. Таким образом, параметры эмбеддингов будут содержать $V \cdot E + E \cdot H$ элементов, что гораздо меньше $V \cdot H$, если $H \gg E$. Авторы выбирают $E = 128$, однако ничего не мешает нам взять любое другое значение. Например, выбрав $E = 64$, мы уменьшим число параметров примерно на 20М.

__Задание 3 (1 балл).__ Напишите класс-обертку над слоем эмбеддингов, который реализует факторизацию на две матрицы, и дообучите факторизованную модель. Заметьте, обе матрицы можно инициализировать с помощью SVD разложения, чтобы начальное приближение было хорошим. Это сэкономит очень много времени на дообучении. С рангом разложения $E = 64$ у вас должно получиться F1 больше 0.87.

In [39]:
# Факторизация word-эмбеддингов BERT на две матрицы (ранг E=64) с инициализацией по SVD
import copy
import torch
import torch.nn as nn
from transformers import DataCollatorForTokenClassification, TrainingArguments, Trainer

class FactorizedWordEmbeddings(nn.Module):
    """
    nn.Embedding(V, E) + nn.Linear(E, H, bias=False) — возвращает [B, T, H],
    чтобы быть взаимозаменяемым с исходными word_embeddings BERT.
    """
    def __init__(self, V: int, E: int, H: int):
        super().__init__()
        self.embedding = nn.Embedding(V, E)
        self.proj = nn.Linear(E, H, bias=False)

    def forward(self, input_ids):
        x = self.embedding(input_ids)   # [B, T, E]
        x = self.proj(x)                # [B, T, H]
        return x

@torch.no_grad()
def factorize_bert_word_embeddings(model: nn.Module, E: int = 64):
    """
    Подменяет bert.embeddings.word_embeddings на факторизованный слой.
    Инициализация по SVD исходной матрицы эмбеддингов W ≈ W1 @ W2.
    """
    # Исходные веса эмбеддингов (V x H)
    orig_word_emb = model.bert.embeddings.word_embeddings
    W = orig_word_emb.weight.detach().cpu()  # [V, H]
    V, H = W.shape

    r = min(E, min(V, H))
    # SVD: W = U @ diag(S) @ Vh
    U, S, Vh = torch.linalg.svd(W, full_matrices=False)
    U_r = U[:, :r]           # [V, r]
    S_r = S[:r]              # [r]
    Vh_r = Vh[:r, :]         # [r, H]
    sqrt_S = torch.sqrt(S_r)

    # W ≈ (U_r * sqrt(S_r)) @ (sqrt(S_r) * Vh_r)
    W1 = U_r * sqrt_S.unsqueeze(0)      # [V, r]
    W2 = sqrt_S.unsqueeze(1) * Vh_r     # [r, H]

    fact = FactorizedWordEmbeddings(V, r, H)
    fact.embedding.weight.copy_(W1)
    fact.proj.weight.copy_(W2.T)

    # Переносим на устройство модели и подменяем слой
    device = orig_word_emb.weight.device
    model.bert.embeddings.word_embeddings = fact.to(device)
    return model

In [40]:
E_rank = 64
model_factorized = copy.deepcopy(trainer.model)
model_factorized = factorize_bert_word_embeddings(model_factorized, E_rank)

n_params_before = sum(p.numel() for p in trainer.model.parameters())
n_params_after = sum(p.numel() for p in model_factorized.parameters())
print("Параметров до факторизации:", n_params_before)
print("Параметров после факторизации:", n_params_after)

print('delta:', n_params_after - n_params_before, 'params')

Параметров до факторизации: 107726601
Параметров после факторизации: 87362569
delta: -20364032 params


In [41]:
# your code here
os.environ["WANDB_DISABLED"] = "true"
os.environ["COMET_LOG_ASSETS"] = "false" #todo

args = TrainingArguments(
    "data/bert-finetuned-ner-factorized",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    # report_to=["comet_ml"],
)

trainer_fact = Trainer(
    model=model_factorized,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=tokenizer,
)

trainer_fact.train()

comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.157500,0.098525,0.951667,0.942147,0.946883,0.987844
2,0.055600,0.096647,0.960761,0.958662,0.959711,0.989463
3,0.032800,0.094071,0.965982,0.959518,0.962739,0.990566


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=2634, training_loss=0.07113703055732976, metrics={'train_runtime': 495.4651, 'train_samples_per_second': 85.017, 'train_steps_per_second': 5.316, 'total_flos': 1125979615856448.0, 'train_loss': 0.07113703055732976, 'epoch': 3.0})

In [42]:
test_metrics(trainer_fact)

TEST metrics


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'precision': 0.9490049751243781,
 'recall': 0.9272569444444444,
 'f1': 0.9380049174569723,
 'accuracy': 0.979908920439325}

In [43]:
del trainer_fact
del model_factorized

### Дистилляция знаний

Дистилляция знаний – это парадигма обучения, в которой знания модели-учителя дистиллируются в модель-ученика. Учеником может быть произвольная модель меньшего размера, решающая ту же задачу, однако обычно ученик имеет ту же архитектуру, что и учитель. При дистилляции используются два функционала ошибки:

1. Стандартная кросс-энтропия.
1. Функция, задающая расстояние между распределениями предсказаний учителя и ученика. Чаще всего используют KL-дивергенцию.

Для того, чтобы распределение предсказаний учителя не было вырожденным, к softmax добавляют температуру больше 1, например, 2 или 5.   
__Важно:__ при делении логитов на температуру значения градиентов уменьшаются в $\tau^2$ раз (проверьте это!). Поэтому для возвращения их в изначальный масштаб ошибку надо домножить на $\tau^2$. Подробнее об этом можно почитать в разделе 2.1 [оригинальной статьи](https://arxiv.org/pdf/1503.02531).

<img src="https://intellabs.github.io/distiller/imgs/knowledge_distillation.png" width="800">

__Задание 4 (3 балла).__ Реализуйте метод дистилляции знаний, изображенный на картинке. Для подсчета ошибки между предсказаниями ученика и учителя используйте KL-дивергенцию [`nn.KLDivLoss(reduction="batchmean")`](https://pytorch.org/docs/stable/generated/torch.nn.KLDivLoss.html) (обратите внимание на вормат ее входов). Для получения итоговой ошибки суммируйте мягкую ошибку с жесткой.   
В качестве учителя используйте дообученный BERT из задания 2. В качестве ученика возьмите необученную модель с размером __не больше 20M__ параметров. Вы можете использовать факторизацию матрицы эмбеддингов для уменьшения числа параметров. Если вы все сделали правильно, то на тестовой выборке вы должны получить значение F1 не меньше 0.7. Вам должно хватить примерно 20к итераций обучения для этого. Если у вас что-то не получается, то можно ориентироваться на статью про [DistilBERT](https://arxiv.org/abs/1910.01108) и на [эту статью](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT).

__Важно:__
* Не забывайте добавлять _warmup_ при обучении ученика.
* Не забывайте переводить учителя в режим _eval_.

In [44]:
from transformers import TrainingArguments, Trainer, infer_device
import torch
import torch.nn as nn
import torch.nn.functional as F

class DistilTrainer(Trainer):
    def __init__(self, teacher_model, student_model, temperature=2.0, lambda_param=0.5,  *args, **kwargs):
        super().__init__(model=student_model, *args, **kwargs)
        self.teacher = teacher_model
        self.student = student_model
        self.loss_function = nn.KLDivLoss(reduction="batchmean")
        device = infer_device()
        self.teacher.to(device)
        self.teacher.eval()
        self.temperature = temperature
        self.lambda_param = lambda_param

    def compute_loss(self, student, inputs, return_outputs=False, num_items_in_batch=None):
        student_output = self.student(**inputs)
        
        with torch.no_grad():
          teacher_output = self.teacher(**inputs)

        soft_teacher = F.softmax(teacher_output.logits / self.temperature, dim=-1)
        soft_student = F.log_softmax(student_output.logits / self.temperature, dim=-1)

        distillation_loss = self.loss_function(soft_student, soft_teacher) * (self.temperature ** 2)
        student_target_loss = student_output.loss

        loss = (1. - self.lambda_param) * student_target_loss + self.lambda_param * distillation_loss
        return (loss, student_output) if return_outputs else loss

In [45]:
# your code here
from transformers import BertConfig, BertForTokenClassification

teacher = trainer.model
teacher.eval()

student_config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=256,
    num_hidden_layers=4,
    num_attention_heads=4,     # 4 * 64 = 256
    intermediate_size=1024,
    max_position_embeddings=512,
    type_vocab_size=len(label2id),
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    num_labels=len(label_names)
)
student = BertForTokenClassification(student_config)

print('Число параметров ученика:', sum(p.numel() for p in student.parameters()))

Число параметров ученика: 10718217


In [46]:
args_distill = TrainingArguments(
    "data/bert-finetuned-ner-distill",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    num_train_epochs=10,
    weight_decay=0.01,

    warmup_ratio=0.1,
    lr_scheduler_type='cosine',
)

trainer_distill = DistilTrainer(
    teacher_model=teacher,
    student_model=student,
    temperature=2.0,
    
    args=args_distill,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=tokenizer,
)

trainer_distill.train()

comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,82.694300,30.163336,0.579656,0.524455,0.550676,0.901425
2,23.957000,21.438402,0.697373,0.648755,0.672186,0.938173
3,14.987600,18.220110,0.782082,0.717442,0.748368,0.954112
4,11.438600,16.233385,0.798417,0.770894,0.784414,0.958939
5,9.369900,15.097805,0.821657,0.780515,0.800558,0.962368
6,7.527500,14.714479,0.832435,0.805049,0.818513,0.965577
7,6.175100,14.148076,0.827551,0.804255,0.815737,0.965429
8,5.625800,14.020853,0.840013,0.812775,0.826170,0.967784
9,5.383400,14.010098,0.841192,0.812856,0.826781,0.967563
10,5.021900,13.990170,0.840687,0.810258,0.825192,0.967740


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr

TrainOutput(global_step=8780, training_loss=15.52052954324014, metrics={'train_runtime': 691.1309, 'train_samples_per_second': 203.16, 'train_steps_per_second': 12.704, 'total_flos': 139819036225680.0, 'train_loss': 15.52052954324014, 'epoch': 10.0})

In [47]:
test_metrics(trainer_distill)

TEST metrics


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


{'precision': 0.7990405117270789,
 'recall': 0.7710905349794238,
 'f1': 0.7848167539267016,
 'accuracy': 0.9514662548651928}

In [48]:
trainer_distill.save_model('model_distill')

In [49]:
model_distill = BertForTokenClassification.from_pretrained('model_distill')

In [87]:
# !zip -r file.zip /kaggle/working/model_distill

  adding: kaggle/working/model_distill/ (stored 0%)
  adding: kaggle/working/model_distill/model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 7%)
  adding: kaggle/working/model_distill/config.json (deflated 57%)
  adding: kaggle/working/model_distill/training_args.bin (deflated 51%)
  adding: kaggle/working/model_distill/vocab.txt (deflated 49%)
  adding: kaggle/working/model_distill/tokenizer.json (deflated 70%)
  adding: kaggle/working/model_distill/tokenizer_config.json (deflated 75%)
  adding: kaggle/working/model_distill/special_tokens_map.json (deflated 42%)


In [50]:
del trainer
del teacher
del student

# Задания на выбор

Как вы понимаете, есть еще довольно много разных способов уменьшить обученную модель. В этой секции вам предлагается реализовать разные техники на выбор. За каждую из них можно получить разное количество балов в зависимости от сложности. Успешность реализации будет оцениваться как по коду, так и по качеству на тестовой выборке. Все баллы за это дз, которые вы наберете сверх 10, будут считаться бонусными.   <!--  -->
В задании 4 вы обучали модель с ограничением числа параметров в 20М. При реализации техник из этой секции придерживайтесь такого же ограничения. Это позволит честно сравнивать методы между собой и делать правильные выводы. Напишите в отчете обо всем, что вы попробовали.

* __Шеринг весов (1 балл).__ В модификации BERT [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) помимо факторизации эмбеддингов предлагается шерить веса между слоями. То есть разные слои используют одни и те же веса. Такая техника эвивалентна применению одного и того же слоя несколько раз. Она позволяет в несколько раз уменьшить число параметров и не сильно потерять в качестве.
* __Факторизация промежуточных слоев (1 балл).__ Если можно факторизовать матрицу эмбеддингов, то и все остальное тоже можно. Для факторизации слоев существует много разных подходов и выбрать какой-то один сложно. Вы можете вдохновляться [этим списком](https://lechnowak.com/posts/neural-network-low-rank-factorization-techniques/), найти в интернете что-то другое или придумать метод самостоятельно. В любом случае в отчете обоснуйте, почему вы решили сделать так как сделали.
* __Приближение промежуточных слоев (2 балла).__ Мы обсуждали, что помимо приближения выходов модели ученика к выходам модели учителя, можно приближать выходы промежуточных слоев. В [этой работе](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT) подробно написано, как это можно сделать.
* __Прунинг (4 балла).__ В методе [SparseGPT](https://arxiv.org/abs/2301.00774) предлагается подход, удаляющий веса модели один раз после обучения. При этом оказывается возможным удалить до половины всех весов без потери в качестве. Математика, стоящаяя за техникой, довольно сложная, однако общий подход простой – будем удалять веса в каждом слое по отдельности, при удалении части весов слоя, остальные веса будут перенастраиваться так, чтобы общий выход слоя не изменился.
* __Удаление голов (6 баллов).__ В данный момент мы используем все головы внимания, но ряд исследований показывает, что большинство из них можно выбросить без потери качества. В этой [статье](https://arxiv.org/pdf/1905.09418.pdf) предлагается подход, который добавляет гейты к механизму внимания, которые регулируют, какие головы участвуют в слое, а какие – нет. В процессе обучения гейты настраиваются так, чтобы большинство голов не использовалась. В конце обучения неиспользуемые головы можно удалить. За это задание дается много баллов, потому что в методе довольно сложная математика и подход плохо заводится. Если вы решитесь потратить на него свои силы, то в случае неудачи мы дадим промежуточные баллы, опираясь на отчет.   
__Совет:__ во время обучения внимательно следите за поведением гейтов. Если вы все сделали правильно, то они должны зануляться. Однако зануляются они не всегда сразу, им надо дать время и обучать модель подольше.

In [154]:
from transformers import TrainingArguments, Trainer
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertConfig, PreTrainedModel
from transformers.models.bert.modeling_bert import (
    BertEmbeddings, 
    BertLayer, 
    BertPooler,
    BertPreTrainedModel
)
from transformers.modeling_outputs import TokenClassifierOutput

class AlbertStyleEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.shared_layer = BertLayer(config)
        self.num_hidden_layers = config.num_hidden_layers
        
    def forward(
        self,
        hidden_states,
    ):
        for i in range(self.num_hidden_layers):
            layer_outputs = self.shared_layer(
                hidden_states,
            )
            
            hidden_states = layer_outputs[0]
            
        return hidden_states


class AlbertStyleBertForTokenClassification(BertPreTrainedModel):
    """BERT с шерингом весов для Token Classification"""
    
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config
        
        self.embeddings = BertEmbeddings(config)
        self.encoder = AlbertStyleEncoder(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        
        # Initialize weights
        self.post_init()
    
    def forward(
        self,
        input_ids=None,
        labels=None,
        attention_mask=None,
    ):
        
        input_shape = input_ids.size()
        embedding_output = self.embeddings(
            input_ids=input_ids,
        )
        encoder_outputs = self.encoder(
            embedding_output,
        )
        
        sequence_output = self.dropout(encoder_outputs)
        logits = self.classifier(sequence_output)
        
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        # print(logits.shape)
        # print(encoder_outputs.shape)
        
        return TokenClassifierOutput(
            loss=loss,
            logits=logits,
            # hidden_states=encoder_outputs,
            # attentions=attention_mask
        )

In [155]:
weight_sharing_config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=512,
    num_hidden_layers=3,
    num_attention_heads=4,
    intermediate_size=1024,
    max_position_embeddings=512,
    type_vocab_size=len(label2id),
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    num_labels=len(label_names)
)

model_weight_sharing = AlbertStyleBertForTokenClassification(weight_sharing_config)

total_params = sum(p.numel() for p in model_weight_sharing.parameters())
print(f"Шеринг весов, параметров: {total_params:,}")


args_weight_sharing = TrainingArguments(
    "data/bert-finetuned-ner-weightsharing",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-4,
    num_train_epochs=5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type='cosine',
    # report_to=["comet_ml"],
)

trainer_weight_sharing = Trainer(
    model=model_weight_sharing,
    args=args_weight_sharing,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=tokenizer,
)


trainer_weight_sharing.train()

comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Шеринг весов, параметров: 17,221,129


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.649000,0.289280,0.747895,0.614841,0.674873,0.938688
2,0.187400,0.253121,0.765914,0.730133,0.747596,0.950374
3,0.093300,0.196570,0.846581,0.808459,0.827081,0.967416
4,0.041400,0.189352,0.857865,0.828562,0.842959,0.970389
5,0.019600,0.204857,0.860391,0.832627,0.846281,0.971096


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers

TrainOutput(global_step=4390, training_loss=0.1698837019587975, metrics={'train_runtime': 157.4185, 'train_samples_per_second': 445.977, 'train_steps_per_second': 27.887, 'total_flos': 46570621521600.0, 'train_loss': 0.1698837019587975, 'epoch': 5.0})

In [156]:
test_metrics(trainer_weight_sharing)

TEST metrics


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'precision': 0.7963752665245203,
 'recall': 0.7641943734015345,
 'f1': 0.7799530148786217,
 'accuracy': 0.9510565544192496}

* > Параметров чуть меньше, чем в задании 4, качество сравнимое,учится быстро (5 эпох за 3 минуты, здорово!). Метрики даже почти не изменились, хотя учила всего 3 эпохи. Параметры особо не меняла - и конфиг, и TrainingArguments почти такие же оставила, просто подогнала параметры модели под число параметров. Ну и lr тоже такой, чтобы модель не застревала после 2 эпохи.